In [1]:
from __future__ import division

from pyqtgraph.Qt import QtGui, QtCore
import numpy as np
from scipy.signal import filtfilt
from numpy import nonzero, diff

import pyqtgraph as pg

#!/usr/bin/env python

from recorder import SoundCardDataSource
w = np.load('pretrained.npz')['w']
b = np.load('pretrained.npz')['b']

act_limit=3
pg.setConfigOption('background', 'w')
pg.setConfigOption('foreground', 'k')
# Based on function from numpy 1.8
def rfftfreq(n, d=1.0):
    """
    Return the Discrete Fourier Transform sample frequencies
    (for usage with rfft, irfft).

    The returned float array `f` contains the frequency bin centers in cycles
    per unit of the sample spacing (with zero at the start). For instance, if
    the sample spacing is in seconds, then the frequency unit is cycles/second.

    Given a window length `n` and a sample spacing `d`::

    f = [0, 1, ..., n/2-1, n/2] / (d*n) if n is even
    f = [0, 1, ..., (n-1)/2-1, (n-1)/2] / (d*n) if n is odd

    Unlike `fftfreq` (but like `scipy.fftpack.rfftfreq`)
    the Nyquist frequency component is considered to be positive.

    Parameters
    ----------
    n : int
    Window length.
    d : scalar, optional
    Sample spacing (inverse of the sampling rate). Defaults to 1.

    Returns
    -------
    f : ndarray
    Array of length ``n//2 + 1`` containing the sample frequencies.
    """
    if not isinstance(n, int):
        raise ValueError("n should be an integer")
    val = 1.0/(n*d)
    N = n//2 + 1
    results = np.arange(0, N, dtype=int)
    return results * val


def fft_slices(x):
    Nslices, Npts = x.shape
    window = np.hanning(Npts)

    # Calculate FFT
    fx = np.fft.rfft(window[np.newaxis, :] * x, axis=1)

    # Convert to normalised PSD
    Pxx = abs(fx)**2 / (np.abs(window)**2).sum()

    # Scale for one-sided (excluding DC and Nyquist frequencies)
    Pxx[:, 1:-1] *= 2

    # And scale by frequency to get a result in (dB/Hz)
    # Pxx /= Fs
    return Pxx ** 0.5


def find_peaks(Pxx):
    # filter parameters
    b, a = [0.01], [1, -0.99]
    Pxx_smooth = filtfilt(b, a, abs(Pxx))
    peakedness = abs(Pxx) / Pxx_smooth

    # find peaky regions which are separated by more than 10 samples
    peaky_regions = nonzero(peakedness > 1)[0]
    edge_indices = nonzero(diff(peaky_regions) > 10)[0]  # RH edges of peaks
    edges = [0] + [(peaky_regions[i] + 5) for i in edge_indices]
    if len(edges) < 2:
        edges += [len(Pxx) - 1]

    peaks = []
    for i in range(len(edges) - 1):
        j, k = edges[i], edges[i+1]
        peaks.append(j + np.argmax(peakedness[j:k]))
    return peaks


def fft_buffer(x):
    window = np.hanning(x.shape[0])

    # Calculate FFT
    fx = np.fft.rfft(window * x)

    # Convert to normalised PSD
    Pxx = abs(fx)**2 / (np.abs(window)**2).sum()

    # Scale for one-sided (excluding DC and Nyquist frequencies)
    Pxx[1:-1] *= 2

    # And scale by frequency to get a result in (dB/Hz)
    # Pxx /= Fs
    return Pxx ** 0.5


class LiveFFTWindow(pg.GraphicsWindow):
    def __init__(self, recorder):
        super(LiveFFTWindow, self).__init__(title="Live FFT")
        self.recorder = recorder
        self.paused = False
        self.logScale = False
        self.showPeaks = False
        self.downsample = True

        # Setup plots
        self.p1 = self.addPlot()
        self.p1.setLabel('bottom', 'Time', 's')
        self.p1.setLabel('left', 'Amplitude')
        self.p1.setTitle("")
        self.ts = self.p1.plot(pen='b')
        self.nextRow()
        self.p2 = self.addPlot()
        self.p2.setLabel('bottom', 'Frequency', 'Hz')
        self.spec = self.p2.plot(pen=(0, 0, 200),
                                 brush=(0, 0, 200),
                                 fillLevel=-100)
        self.nextRow()
        self.p3 = self.addPlot()
        #self.p3.setLabel('bottom', 'Frequency', 'Hz')
        #self.p3.setLabel('left', 'Weight', '')
        self.p3.setTitle('Perceptron weights')
        self.weight = self.p3.plot(pen='b', brush='b')

        self.nextRow()
        self.p4 = self.addPlot()
        #self.p4.setLabel('bottom', 'INPUT', '')
        #self.p4.setLabel('left', 'OUTPUT', '')
        self.p4.setTitle('Activation')
        self.act = self.p4.plot(pen='b', brush='b')

        self.nextRow()
        self.p5 = self.addPlot()
        self.p5.setTitle('Non-Linearity')

        x = np.linspace(-act_limit, act_limit, 1000)
        y = np.tanh(x)
        self.p5.plot(pen='r', brush='r').setData(x=x, y=y)

        #notePen = pg.mkPen((220, 0, 50, 100))
        #self.act_line = self.p4.addLine(x=0, pen=notePen)

        '''
        # Show note lines
            A = 440.0
            notePen = pg.mkPen((0, 200, 50, 100))
            while A < (self.recorder.fs / 2):
                self.p2.addLine(x=A, pen=notePen)
                A *= 2
        '''

        # Lines for marking peaks
        self.peakMarkers = []

        # Data ranges
        self.resetRanges()

        # Timer to update plots
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update)
        interval_ms = 1000 * (self.recorder.chunk_size / self.recorder.fs)
        #print ("Updating graphs every %.1f ms" % interval_ms)
        self.timer.start(interval_ms)

    def resetRanges(self):
        self.timeValues = self.recorder.timeValues
        self.freqValues = rfftfreq(len(self.timeValues),
                                   1./self.recorder.fs)

        self.p1.setRange(xRange=(0, self.timeValues[-1]), yRange=(-1, 1))
        if self.logScale:
            self.p2.setRange(xRange=(0, self.freqValues[-1]),
                             yRange=(-60, 20))
            self.spec.setData(fillLevel=-100)
            self.p2.setLabel('left', 'PSD', 'dB / Hz')
        else:
            self.p2.setRange(xRange=(0, self.freqValues[-1]),
                             yRange=(0, 50))
            self.spec.setData(fillLevel=0)
            self.p2.setLabel('left', 'PSD', '1 / Hz')

    def plotPeaks(self, Pxx):
        # find peaks bigger than a certain threshold
        peaks = [p for p in find_peaks(Pxx) if Pxx[p] > 0.3]

        if self.logScale:
            Pxx = 20*np.log10(Pxx)

        # Label peaks
        old = self.peakMarkers
        self.peakMarkers = []
        for p in peaks:
            if old:
                t = old.pop()
            else:
                t = pg.TextItem(color=(150, 150, 150, 150))
                self.p2.addItem(t)
            self.peakMarkers.append(t)
            t.setText("%.1f Hz" % self.freqValues[p])
            t.setPos(self.freqValues[p], Pxx[p])
        for t in old:
            self.p2.removeItem(t)
            del t

    def update(self):
        if self.paused:
            return
        data = self.recorder.get_buffer()
        weighting = np.exp(self.timeValues / self.timeValues[-1])
        Pxx = fft_buffer(weighting * data[:, 0])

        if self.downsample:
            downsample_args = dict(autoDownsample=False,
                                   downsampleMethod='subsample',
                                   downsample=10)
        else:
            downsample_args = dict(autoDownsample=True)

        self.ts.setData(x=self.timeValues, y=data[:, 0], **downsample_args)
        self.spec.setData(x=self.freqValues,
                          y=(20*np.log10(Pxx) if self.logScale else Pxx))

        pxx = (Pxx - Pxx.min()) 
        pxx /= pxx.max()
        # Show note lines


        notePen = pg.mkPen((0, 200, 50, 100))
        a = np.dot(w, Pxx) + b
        self.weight.setData(x=self.freqValues, y=w)
        print(a)
        a = max(-act_limit, min(a, act_limit))
        x = np.linspace(-act_limit, act_limit, len(w))
        y = np.tanh(x)
        A = np.ones(len(x))*0

        idx = (np.abs(x-a)).argmin()
        #print(idx)
        if idx == 0:
            A[5:15] = 1
        elif idx == len(A) - 1:
            A[-30:-10] = 1
        else:
            A[idx-10: idx+10] = 1
        self.act.setData(x=x, y=A)
        if self.showPeaks:
            self.plotPeaks(Pxx)

    def keyPressEvent(self, event):
        text = event.text()
        if text == " ":
            self.paused = not self.paused
            self.p1.setTitle("PAUSED" if self.paused else "")
        elif text == "l":
            self.logScale = not self.logScale
            self.resetRanges()
        elif text == "d":
            self.downsample = not self.downsample
        elif text == "+":
            self.recorder.num_chunks *= 2
            self.resetRanges()
        elif text == "-":
            self.recorder.num_chunks /= 2
            self.resetRanges()
        elif text == "p":
            self.showPeaks = not self.showPeaks
        else:
            super(LiveFFTWindow, self).keyPressEvent(event)

        # Setup plots
#QtGui.QApplication.setGraphicsSystem('opengl')
app = QtGui.QApplication([])
#pg.setConfigOptions(antialias=True)

# Setup recorder
#FS = 12000
#FS = 22000
FS = 44000
recorder = SoundCardDataSource(num_chunks=3,
                               sampling_rate=FS,
                               chunk_size=4*1024)
win = LiveFFTWindow(recorder)
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

[-45.86175537]
[-45.86175537]
[-28.54416656]
[-19.35538673]
[-19.35538673]
[-9.97282791]
[-7.34131575]
[-4.83710909]
[-4.83710909]
[-2.81880116]
[-2.81880116]
[-4.41250372]
[-4.54399252]
[-3.86725903]
[-3.86725903]
[-3.86147809]
[-3.42450595]
[-3.42450595]
[-3.78420424]
[-3.78420424]
[-2.6797545]
[-3.92610335]
[-3.92610335]
[-3.40087891]
[-3.40087891]
[-3.62099051]
[-3.47590232]
[-3.47590232]
[-4.9784832]
[-4.9784832]
[-4.53193617]
[-4.53193617]
[-4.38549089]
[-4.31424332]
[-4.31424332]
[-3.41571999]
[-4.23276901]
[-3.70784712]
[-3.70784712]
[-3.3428092]
[-3.3428092]
[-3.41841316]
[-3.83444762]
[-3.83444762]
[-3.76185298]
[-2.69090557]
[-2.69090557]
[-3.78007293]
[-3.19655061]
[-3.19655061]
[-4.43437195]
[-4.43437195]
[-3.74480247]
[-3.2730329]
[-4.37656689]
[-4.37656689]
[-4.30045605]
[-4.05739403]
[-4.05739403]
[-3.23692226]
[-3.06652236]
[-2.71544838]
[-2.71544838]
[-2.8463192]
[-3.08590531]
[-3.08590531]
[-3.42034936]
[-3.43939972]
[-3.49321485]
[-3.49321485]
[-3.5931499]
[-3.59314

[-2.880023]
[-2.82068706]
[-2.82068706]
[-3.44564724]
[-4.15544605]
[-4.15544605]
[-3.46977305]
[-3.27196717]
[-3.6569488]
[-3.6569488]
[-3.48196578]
[-3.48196578]
[-3.68690753]
[-3.98490572]
[-3.98490572]
[-3.56613874]
[-3.12676096]
[-3.12676096]
[-3.89489818]
[-3.26268578]
[-3.26268578]
[-2.48981667]
[-2.48981667]
[-2.9039371]
[-2.69822764]
[-4.18370152]
[-4.18370152]
[-5.15195847]
[-3.02207661]
[-3.02207661]
[-2.69905281]
[-3.09186006]
[-2.84938955]
[-2.84938955]
[-2.67754984]
[-2.80607605]
[-2.80607605]
[-3.21632266]
[-3.21632266]
[-3.71354675]
[-3.66156101]
[-3.66156101]
[-2.6263237]
[-3.22135258]
[-3.22135258]
[-2.90731788]
[-2.90731788]
[-3.1273551]
[-3.35978413]
[-3.35978413]
[-4.64639378]
[-4.64639378]
[-4.38505363]
[-4.38505363]
[-4.50766182]
[-3.94211435]
[-3.94211435]
[-3.63491249]
[-3.57741499]
[-3.87047553]
[-3.87047553]
[-2.7506001]
[-6.83043528]
[-6.83043528]
[-3.65551233]
[-3.02661347]
[-2.82285786]
[-2.82285786]
[-2.90168595]
[-3.78099608]
[-4.11861706]
[-4.6805706]
[

[-3.0574789]
[-3.74471211]
[-3.4799943]
[-3.4799943]
[-3.92746472]
[-3.92746472]
[-2.82625175]
[-5.00122929]
[-5.00122929]
[-4.80399799]
[-2.39583635]
[-2.39583635]
[-3.32562017]
[-3.32562017]
[-2.50476027]
[-2.50476027]
[-3.14429259]
[-3.14429259]
[-3.56223965]
[-3.72199035]
[-3.72199035]
[-3.26377773]
[-3.26377773]
[-3.62028027]
[-3.62028027]
[-3.27697349]
[-3.05645514]
[-3.05645514]
[-4.16703081]
[-4.03673744]
[-3.80450964]
[-3.80450964]
[-3.56804633]
[-2.78436375]
[-3.17934322]
[-3.17934322]
[-3.62470508]
[-2.93016434]
[-2.93016434]
[-3.55147839]
[-3.76510906]
[-3.79718637]
[-3.79718637]
[-3.73822188]
[-3.73822188]
[-2.99322605]
[-4.12727737]
[-4.12727737]
[-4.32886744]
[-2.48545599]
[-2.48545599]
[-2.59780979]
[-2.53728771]
[-2.53728771]
[-2.94932604]
[-2.94932604]
[-3.76892352]
[-4.53777456]
[-4.53777456]
[-4.28861523]
[-3.01669073]
[-3.01669073]
[-3.74425292]
[-3.74425292]
[-6.0899725]
[-4.10316563]
[-4.10316563]
[-6.46100283]
[-7.77861166]
[-7.77861166]
[-4.5107708]
[-3.9491138

[-3.85930824]
[-4.58182907]
[-4.58182907]
[-3.66737318]
[-4.58602524]
[-4.34482384]
[-4.34482384]
[-5.03384304]
[-6.42138863]
[-6.42138863]
[-6.29472637]
[-5.82563782]
[-5.95362949]
[-5.95362949]
[-4.68250847]
[-4.68250847]
[-4.38991928]
[-5.40497446]
[-5.37323427]
[-5.37323427]
[-4.38777018]
[-5.81849527]
[-5.81849527]
[-4.28153896]
[-4.94089746]
[-4.94089746]
[-4.4982543]
[-4.71438408]
[-4.71438408]
[-3.90619206]
[-3.90619206]
[-4.11918163]
[-5.24917507]
[-5.24917507]
[-5.64355564]
[-4.59468555]
[-3.258322]
[-3.258322]
[-3.23946881]
[-3.18279386]
[-3.18279386]
[-4.0571785]
[-9.17856789]
[-9.17856789]
[-11.47942543]
[-4.43736458]
[-4.43736458]
[-5.67107439]
[-5.67107439]
[-5.34074593]
[-5.9075408]
[-5.9075408]
[-6.25481462]
[-5.6712656]
[-5.6712656]
[-5.86000061]
[-5.86000061]
[-4.24265194]
[-5.42591095]
[-5.42591095]
[-5.41786575]
[-4.30686235]
[-4.30686235]
[-4.34654999]
[-3.44687247]
[-2.8066771]
[-2.8066771]
[-3.05315852]
[-3.05315852]
[-3.17530489]
[-3.22682095]
[-3.22682095]
[-2

[-4.3336854]
[-4.3336854]
[-4.16241074]
[-4.08557892]
[-4.08557892]
[-3.05463171]
[-3.05463171]
[-3.28605247]
[-3.28605247]
[-3.71157885]
[-2.56552649]
[-2.56552649]
[-3.25985146]
[-3.25985146]
[-3.3942759]
[-3.3942759]
[-3.32580662]
[-3.32580662]
[-3.16514087]
[-2.81164575]
[-2.81164575]
[-3.00546193]
[-3.10456729]
[-3.10456729]
[-3.58958149]
[-2.37134647]
[-2.37134647]
[-3.15246487]
[-3.66679549]
[-3.72032833]
[-3.72032833]
[-3.81434441]
[-2.69318962]
[-2.69318962]
[-2.76319385]
[-2.76319385]
[-2.62834167]
[-2.62834167]
[-2.37318516]
[-2.37318516]
[-2.78450227]
[-3.95433402]
[-3.95433402]
[-2.95581532]
[-2.95581532]
[-2.41137409]
[-2.41137409]
[-2.50220728]
[-2.96209216]
[-2.96209216]
[-4.16107559]
[-4.60197687]
[-4.60197687]
[-3.82615447]
[-3.82615447]
[-2.98298883]
[-6.62960005]
[-6.62960005]
[-4.7364502]
[-4.29941463]
[-4.29941463]
[-3.8224659]
[-3.81277394]
[-3.10656571]
[-3.10656571]
[-2.70923734]
[-3.31391907]
[-3.31391907]
[-3.74452376]
[-3.4030087]
[-3.4030087]
[-3.60915637]


...

[-3.35697532]
[-2.30439091]
[-3.10019469]
[-3.10019469]
[-3.16134191]
[-3.16134191]
[-3.18392873]
[-2.81959105]
[-2.81959105]
[-3.65083027]
[-3.65083027]
[-4.49195576]
[-4.49195576]
[-3.65769196]
[-3.65769196]
[-3.64122891]
[-3.64122891]
[-3.08836889]
[-3.28514457]
[-3.28514457]
[-2.71031618]
[-2.71031618]
[-3.22717977]
[-3.22717977]
[-2.56000996]
[-2.56000996]
[-3.24462342]
[-3.24462342]
[-5.86502838]
[-5.86502838]
[-3.25162196]
[-3.25162196]
[-2.40745354]
[-2.40745354]
[-3.41660714]
[-3.41660714]
[-2.63080406]
[-2.63080406]
[-2.37321424]
[-2.37321424]
[-2.73209262]
[-2.73209262]
[-3.65238738]
[-3.65238738]
[-4.25277424]
[-4.25277424]
[-4.3721776]
[-2.96045017]
[-2.96045017]
[-2.67756081]
[-2.67756081]
[-2.41390276]
[-2.41390276]
[-2.58968997]
[-2.58968997]
[-2.9458158]
[-2.9458158]
[-2.661834]
[-2.661834]
[-2.98470521]
[-2.61301994]
[-2.72377753]
[-2.72377753]
[-2.53102827]
[-2.53102827]
[-2.5385313]
[-2.5385313]
[-2.77801561]
[-2.77801561]
[-2.7297647]
[-2.7297647]
[-2.60732651]
[-2

[-2.89316559]
[-2.89316559]
[-3.19338751]
[-3.19338751]
[-2.30425882]
[-2.30425882]
[-3.47033954]
[-3.47033954]
[-3.17516255]
[-2.68311357]
[-2.68311357]
[-3.29777431]
[-3.29777431]
[-4.11690044]
[-4.11690044]
[-2.98563433]
[-2.98563433]
[-3.12076402]
[-3.12076402]
[-3.01963568]
[-3.01963568]
[-3.00697255]
[-3.02174067]
[-3.02174067]
[-3.85564423]
[-3.85564423]
[-4.05535889]
[-4.05535889]
[-3.87346959]
[-3.87346959]
[-3.38655162]
[-3.54241371]
[-3.54241371]
[-3.64859533]
[-3.64859533]
[-2.90313268]
[-2.99444675]
[-2.99444675]
[-3.14137459]
[-3.14137459]
[-2.57199192]
[-2.57199192]
[-2.84478331]
[-2.84478331]
[-2.70513654]
[-3.30113387]
[-3.30113387]
[-2.83571768]
[-2.83571768]
[-3.06745386]
[-3.06745386]
[-3.95618105]
[-3.95618105]
[-4.70298767]
[-4.70298767]
[-3.49816489]
[-3.49816489]
[-2.98997498]
[-2.98997498]
[-3.17309952]
[-2.69630885]
[-2.69630885]
[-2.85721684]
[-2.85721684]
[-2.79848433]
[-3.01157904]
[-3.01157904]
[-2.52591324]
[-2.52591324]
[-3.28773499]
[-3.28773499]
[-3.13

[-11.43748665]
[-5.39629459]
[-5.39629459]
[-8.87073898]
[-8.87073898]
[-8.83679581]
[-8.83679581]
[-5.78456736]
[-5.78456736]
[-4.00825453]
[-3.89602208]
[-3.89602208]
[-3.50200963]
[-3.50200963]
[-3.2916193]
[-3.2916193]
[-3.57915092]
[-3.62506437]
[-3.62506437]
[-5.0284214]
[-5.0284214]
[-3.66019368]
[-3.66019368]
[-4.3395009]
[-4.31537342]
[-4.31537342]
[-5.04100561]
[-5.04100561]
[-5.78606129]
[-5.78606129]
[-4.80204678]
[-4.80204678]
[-4.410676]
[-4.410676]
[-4.75593376]
[-4.75593376]
[-12.10124207]
[-12.10124207]
[-114.75861359]
[-8.31065369]
[-8.31065369]
[-24.81982613]
[-24.81982613]
[-15.20748711]
[-15.20748711]
[-21.99176598]
[-12.55640984]
[-12.55640984]
[-4.72825003]
[-4.72825003]
[-8.53742599]
[-8.53742599]
[-49.91893768]
[-11.51820755]
[-11.51820755]
[-9.92058945]
[-9.92058945]
[-5.28056717]
[-5.28056717]
[-4.31552696]
[-4.31552696]
[-5.33542538]
[-5.33542538]
[-7.38249636]
[-7.38249636]
[-4.6656847]
[-9.23401833]
[-9.23401833]
[-13.72187042]
[-13.72187042]
[-55.52899933

[-11.64515305]
[-14.30905724]
[-14.30905724]
[-13.18526077]
[-13.18526077]
[-10.40166664]
[-10.40166664]
[-8.78686523]
[-8.78686523]
[-8.18317986]
[-8.18317986]
[-7.00372839]
[-9.31986427]
[-9.31986427]
[-14.87898254]
[-14.87898254]
[-9.37568665]
[-9.37568665]
[-9.90206146]
[-9.90206146]
[-10.44590759]
[-10.44590759]
[-10.8038311]
[-10.8038311]
[-16.40809631]
[-16.40809631]
[-19.00712013]
[-12.88789749]
[-12.88789749]
[-22.07235336]
[-22.07235336]
[-13.12815285]
[-11.79726791]
[-11.79726791]
[-12.3974514]
[-12.3974514]
[-12.27873611]
[-12.27873611]
[-10.4682827]
[-10.4682827]
[-10.28385353]
[-10.28385353]
[-11.02727509]
[-12.8750515]
[-12.8750515]
[-10.04389381]
[-10.04389381]
[-8.50283241]
[-8.50283241]
[-14.85531044]
[-14.85531044]
[-10.34471893]
[-10.34471893]
[-14.62903404]
[-14.62903404]
[-12.92653084]
[-12.92653084]
[-11.36026573]
[-11.36026573]
[-12.77033806]
[-12.77033806]
[-9.15610123]
[-9.15610123]
[-10.29021645]
[-10.29021645]
[-9.39608002]
[-9.39608002]
[-8.62284184]
[-8.62

[-2.86920619]
[-2.86920619]
[-2.67144489]
[-2.67144489]
[-3.6908505]
[-3.6908505]
[-2.86678004]
[-2.86678004]
[-5.18211746]
[-5.18211746]
[-4.99748993]
[-4.99748993]
[-6.29096413]
[-6.29096413]
[-6.09281874]
[-11.48940277]
[-11.48940277]
[-9.6400032]
[-9.6400032]
[-5.20240641]
[-5.20240641]
[-4.65000248]
[-4.65000248]
[-8.52007103]
[-8.52007103]
[-9.29807472]
[-9.29807472]
[-30.60113907]
[-30.60113907]
[-139.98869324]
[-139.98869324]
[-24.8817215]
[-24.8817215]
[-10.06996727]
[-10.06996727]
[-6.82795668]
[-6.82795668]
[-16.28445816]
[-16.28445816]
[-15.96172142]
[-15.96172142]
[-14.52143097]
[-9.92319107]
[-9.92319107]
[-9.21817398]
[-9.21817398]
[-13.07667351]
[-13.07667351]
[-12.34688568]
[-12.34688568]
[-6.52383518]
[-6.52383518]
[-14.04829979]
[-14.04829979]
[-39.75903702]
[-39.75903702]
[-26.15386963]
[-19.67093468]
[-13.76768684]
[-13.76768684]
[-10.02350044]
[-10.02350044]
[-13.13950729]
[-13.13950729]
[-11.68701363]
[-11.68701363]
[-7.26356411]
[-7.26356411]
[-7.44264364]
[-7.4